# Wav2Vec2.0 Pretraining

Trying to reproduce https://github.com/pytorch/fairseq/blob/master/examples/wav2vec/README.md


# Install all prerequirements

In [1]:
import torch
 
torch.cuda.get_device_name(device=None)

'Tesla V100-SXM2-16GB'

In [2]:
!pip install git+https://github.com/huggingface/transformers.git git+https://github.com/pytorch/fairseq pyarrow soundfile torchaudio librosa

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-wpgouz6r
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-wpgouz6r
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/pytorch/fairseq to /tmp/pip-req-build-sgv8albk
  Running command git clone -q https://github.com/pytorch/fairseq /tmp/pip-req-build-sgv8albk
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2213074 sha256=9076f554adb204f368c64a95362c2c621d648b50935acc8267426ac3b9866d7f
  Stored in directory: /tmp/pip-ephem-wheel-cache-0s0gx0bw/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0ed

# Install APEX for mixed precision Fairseq trainings

In [3]:
%%shell
rm -f /usr/local/cuda
ln -sv /usr/local/cuda-10.1 /usr/local/cuda
git clone https://github.com/NVIDIA/apex
cd apex
git reset --hard b5eb38dbf7accc24bd872b3ab67ffc77ee858e62
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" \
  --global-option="--deprecated_fused_adam" --global-option="--xentropy" \
  --global-option="--fast_multihead_attn" ./
cd ..

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
                     from apex/contrib/csrc/multihead_attn/self_multihead_attn_bias_additive_mask.cpp:1:
    /usr/local/lib/python3.7/dist-packages/torch/include/ATen/core/TensorBody.h:303:30: note: declared here
       DeprecatedTypeProperties & type() const {
                                  ^~~~
    In file included from /usr/local/lib/python3.7/dist-packages/torch/include/c10/core/Device.h:5:0,
                     from /usr/local/lib/python3.7/dist-packages/torch/include/c10/core/Allocator.h:6,
                     from /usr/local/lib/python3.7/dist-packages/torch/include/ATen/ATen.h:7,
                     from /usr/local/lib/python3.7/dist-packages/torch/include/torch/csrc/api/include/torch/types.h:3,
                     from /usr/local/lib/python3.7/dist-packages/torch/include/torch/csrc/api/include/torch/data/dataloader_options.h:4,
                     from /usr/local/lib/python3.7/dist-packages/torch/include/

# Gather the data

In [ ]:
%%shell
wget https://os.unil.cloud.switch.ch/fma/fma_large.zip

--2021-04-30 19:21:28--  https://os.unil.cloud.switch.ch/fma/fma_large.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.16, 2001:620:5ca1:201::214
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100306112191 (93G) [application/zip]
Saving to: ‘fma_large.zip’

fma_large.zip       100%[===================>]  93.42G  25.6MB/s    in 62m 29s 

2021-04-30 20:23:58 (25.5 MB/s) - ‘fma_large.zip’ saved [100306112191/100306112191]



In [ ]:
from zipfile import ZipFile
from tqdm import tqdm

with ZipFile('fma_large.zip', 'r') as zip_file:
    for file in tqdm(iterable=zip_file.namelist(), total=len(zip_file.namelist())):
        zip_file.extract(member=file, path='/content/drive/MyDrive/musicology-dataset/')

100%|██████████| 106576/106576 [2:48:56<00:00, 10.51it/s]


# Preprocess the data - wav2vec2 is strict about keeping 16k framerate

In [ ]:
!sudo apt-get install sox libsox-fmt-mp3
! pip install sox pqdm

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libid3tag0 libmad0 libmagic-mgc libmagic1 libopencore-amrnb0
  libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libid3tag0 libmad0 libmagic-mgc libmagic1 libopencore-amrnb0
  libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox-fmt-mp3 libsox3
  sox
0 upgraded, 11 newly installed, 0 to remove and 34 not upgraded.
Need to get 872 kB of archives.
After this operation, 7,087 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrnb0 amd64 0.1.3-2.1 [92.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrwb0 a

In [ ]:
from os import listdir
from os.path import isfile, join
from tqdm.notebook import tqdm
import sox
from pqdm.processes import pqdm

def convert(file):
  resampler = sox.Transformer()
  resampler.rate(16000, quality='h')
  return resampler.build_file(f'{in_path}/{file}', f'{out_path}/{file}')

for num in range(1, 156):
  num = str(num)
  if len(num)==1:
    num = "00"+num
  if len(num)==2:
    num = "0"+num
  print(num)
  in_path = f"/content/drive/MyDrive/musicology-dataset/fma_large/{num}"
  out_path = "/content/drive/MyDrive/musicology-dataset-downsampled"
  files_zero = [f for f in listdir(in_path) if isfile(join(in_path, f))]
  _ = pqdm(files_zero, convert, n_jobs=4)
  

SUBMITTING | :   0%|          | 0/870 [00:00<?, ?it/s]

001


SUBMITTING | : 100%|██████████| 870/870 [00:00<00:00, 10515.92it/s]
PROCESSING | : 100%|██████████| 870/870 [03:47<00:00,  3.82it/s]
COLLECTING | : 100%|██████████| 870/870 [00:00<00:00, 384271.74it/s]
SUBMITTING | :   0%|          | 0/64 [00:00<?, ?it/s]

002


SUBMITTING | : 100%|██████████| 64/64 [00:00<00:00, 2071.50it/s]
PROCESSING | : 100%|██████████| 64/64 [00:16<00:00,  3.89it/s]
COLLECTING | : 100%|██████████| 64/64 [00:00<00:00, 179796.02it/s]
SUBMITTING | :   0%|          | 0/599 [00:00<?, ?it/s]

...

153


SUBMITTING | : 100%|██████████| 363/363 [00:00<00:00, 5300.26it/s]
PROCESSING | : 100%|██████████| 363/363 [01:42<00:00,  3.53it/s]
COLLECTING | : 100%|██████████| 363/363 [00:00<00:00, 305729.39it/s]


154


SUBMITTING | : 100%|██████████| 813/813 [00:00<00:00, 6200.07it/s]
PROCESSING | : 100%|██████████| 813/813 [03:59<00:00,  3.40it/s]
COLLECTING | : 100%|██████████| 813/813 [00:00<00:00, 185657.38it/s]
SUBMITTING | : 0it [00:00, ?it/s]
PROCESSING | : 0it [00:00, ?it/s]
COLLECTING | : 0it [00:00, ?it/s]

155


# Prepare the manifest file for Fairseq training

In [ ]:
import argparse
import glob
import os
import random

from pqdm.processes import pqdm

import sox

valid_percent = 0.05
root = "/content/drive/MyDrive/musicology-dataset-downsampled/"
dest = "/content/drive/MyDrive/musicology-experiment-files/fma_manifests/"
ext = "mp3"

assert valid_percent >= 0 and valid_percent <= 1.0

if not os.path.exists(dest):
    os.makedirs(dest)

dir_path = os.path.realpath(root)
search_path = os.path.join(dir_path, "**/*." + ext)
rand = random.Random(42)

def prepare_line(filename):
    file_path = os.path.realpath(filename)
    frames = sox.file_info.num_samples(filename)
    return "{}\t{}".format(os.path.relpath(file_path, dir_path), frames)

with open(os.path.join(dest, "train.tsv"), "w") as train_f, open(
    os.path.join(dest, "valid.tsv"), "w"
) as valid_f:
    print(dir_path, file=train_f)
    print(dir_path, file=valid_f)

    file_list = glob.iglob(search_path, recursive=True)
    out = pqdm(file_list, prepare_line, n_jobs=64)

    for line in out:
        dest = train_f if rand.random() > valid_percent else valid_f
        print(
            line, file=dest
        )



PROCESSING | : 100%|█████████▉| 105935/106085 [23:05<00:03, 45.48it/s]

PROCESSING | : 100%|█████████▉| 105942/106085 [23:05<00:02, 50.41it/s]

PROCESSING | : 100%|█████████▉| 105979/106085 [23:05<00:01, 66.24it/s]

PROCESSING | : 100%|█████████▉| 105992/106085 [23:05<00:01, 73.78it/s]

PROCESSING | : 100%|█████████▉| 106004/106085 [23:06<00:01, 41.07it/s]

PROCESSING | : 100%|█████████▉| 106024/106085 [23:06<00:01, 50.08it/s]

PROCESSING | : 100%|█████████▉| 106061/106085 [23:06<00:00, 62.63it/s]

PROCESSING | : 100%|█████████▉| 106072/106085 [23:07<00:00, 44.34it/s]

PROCESSING | : 100%|██████████| 106085/106085 [23:07<00:00, 76.45it/s]


COLLECTING | :   0%|          | 0/106085 [00:00<?, ?it/s]

COLLECTING | :  44%|████▎     | 46369/106085 [00:00<00:00, 463689.34it/s]

COLLECTING | : 100%|██████████| 106085/106085 [00:00<00:00, 439503.34it/s]


# Hack: by default Fairseq accepts .flac only. We want .mp3 so it's faster to change the library code than to convert everything

In [ ]:
# Inject the improved raw_audio_dataset to fairseq in order to read mp3
!cp /content/drive/MyDrive/musicology-experiment-files/raw_audio_dataset.py /usr/local/lib/python3.7/dist-packages/fairseq/data/audio/raw_audio_dataset.py

# Install https://wandb.ai/ training tracker to track the training online

In [ ]:
!pip install wandb
!wandb login

     |████████████████████████████████| 2.1MB 13.9MB/s 
     |████████████████████████████████| 163kB 46.2MB/s 
     |████████████████████████████████| 133kB 46.8MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=baaec296c9615adaa51695d7012e7a63dde4a6b227097929a95a197be4da7ef5
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=7e899604a5423662fa780e51d536c3e2b3a5742a6d2634c88dc9cd994f589c80
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: A

# Pre-train the model in the semi-supervised way

We ran 2 full epochs (~16h)

Final training log is available here: https://wandb.ai/glorf/mel2vec/runs/1bid19sm?workspace=user-glorf

Model can be downloaded from Hf Hub: https://huggingface.co/mbien/fma2vec

In [ ]:
%cd /content/drive/MyDrive/musicology-experiment-files

!fairseq-hydra-train \
    distributed_training.distributed_world_size=1 \
    +optimization.update_freq='[64]' \
    task.data=/content/drive/MyDrive/musicology-experiment-files/fma_manifests \
    --config-dir /content/drive/MyDrive/musicology-experiment-files/config \
    --config-name wav2vec2_base_librispeech

/content/drive/MyDrive/musicology-experiment-files
[2021-05-01 22:09:41,107][fairseq_cli.train][INFO] - {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 200, 'log_format': 'json', 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': 'mel2vec', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': Non

# Fine-tuning
Now we only have the bare pretrained model with some musical insight

Let's create a model which can be used for the actual task on top of that

# We're changing the library - bye Fairseq, welcome Huggingface Transformers!
Let's convert the model

In [ ]:
# 1) create a fitting HF config
from transformers import Wav2Vec2Config
config = Wav2Vec2Config.from_pretrained('facebook/wav2vec2-base-960h')
config.save_pretrained('./fma2vec')
# 2) Then run convert (it's important that you pass --not_finetuned python)
!python  /usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/convert_wav2vec2_original_pytorch_checkpoint_to_pytorch.py --pytorch_dump_folder /content/drive/MyDrive/musicology-experiment-files/fma2vec --checkpoint_path /content/drive/MyDrive/musicology-experiment-files/outputs/2021-05-01/22-09-40/checkpoints/checkpoint_best.pt --config_path /content/fma2vec/config.json --not_finetuned

2021-05-02 16:06:48.280898: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
loading configuration file /content/fma2vec/config.json
Model config Wav2Vec2Config {
  "activation_dropout": 0.1,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "group",
  "feat_proj_dropout": 0.1,
  "final_dropout": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout": 0.1,
  "hidden_dr

# Create a fine-tuning pipeline

## First we need to get popularity scores calculated in Milestone 3 part 1
Also, some files are apparently missing on our drive.

Let's not investigate it any further (time constraints) and just validate it according to Fairseq pretraining manifests


In [7]:
import pandas as pd

train = pd.read_csv("pop_train.csv") # Files generated by milestone3.ipynb
test = pd.read_csv("pop_test.csv")

manifest_train = pd.read_csv("/content/drive/MyDrive/musicology-experiment-files/fma_manifests/train.tsv", sep="\t")
manifest_valid = pd.read_csv("/content/drive/MyDrive/musicology-experiment-files/fma_manifests/valid.tsv", sep="\t")

manifests = pd.concat([manifest_train, manifest_valid])
manifests = manifests.reset_index()["index"].str.strip(".mp3").astype(int)

train = train[train.track_id.isin(manifests)]
test = test[test.track_id.isin(manifests)]
minimum = min(train.pop_score.min(), test.pop_score.min())
maximum = max(train.pop_score.max(), test.pop_score.max())

print(minimum, maximum)

-3.3939661205367715 5.098214985136099


# Rescale the scores to fit in (-1, 1) limits
It's better for MSE to have well regulated limits

In [3]:
train.pop_score = (train.pop_score - minimum)/(maximum-minimum)
train.pop_score = (train.pop_score*2)-1

test.pop_score = (test.pop_score - minimum)/(maximum-minimum)
test.pop_score = (test.pop_score*2)-1
train.head()

,track_id,pop_score
0,31464,-0.455719
1,8271,-0.433040
2,62870,-0.362632
4,124434,0.426293
5,140109,-0.192201


## Create a PyTorch dataset
It loads and preprocesses the files from the Disk and provides the raw waveform together with the expected popularity score to the model


In very rare cases (~5 per epoch), the file loading fails, therefore the exception is handled by loading the file with the previous index instead

In [4]:
from tqdm import tqdm
import torchaudio
from transformers import Wav2Vec2FeatureExtractor
import torch.nn.functional as F


feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, max_length=500000)

class PopularityDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        track_id = str(int(row.track_id)).zfill(6)
        audio_sample = []
        try:
          audio_sample, _ = torchaudio.load(f"/content/drive/MyDrive/musicology-dataset-downsampled/{track_id}.mp3")
        except Exception:
          print("Broken file found, fixing with prev file sampling")
          row = self.dataframe.iloc[index-1]
          track_id = str(int(row.track_id)).zfill(6)
          audio_sample, _ = torchaudio.load(f"/content/drive/MyDrive/musicology-dataset-downsampled/{track_id}.mp3")
        audio_sample = torch.mean(audio_sample, dim=0, keepdim=True)
        audio_sample = audio_sample.squeeze().numpy()
        audio_sample = feature_extractor(audio_sample, sampling_rate=16000, return_tensors="pt", padding='max_length', max_length=480000).input_values
        return {
            "input_ids": audio_sample.flatten()[:480000],
            "labels": torch.tensor(row.pop_score).float(),
        }

train_ds = PopularityDataset(train)
test_ds = PopularityDataset(test)

# The model implementation

We follow the BertForSentenceClassification approach
The first output of the model is changed to serve as a 768-values full-context output. These values are passed through a single linear layer, with 20% dropout during training, and are expected to return one value - our popularity score.

In [5]:
import torch.nn as nn
from transformers import Wav2Vec2PreTrainedModel, Wav2Vec2Model
from transformers.modeling_outputs import SequenceClassifierOutput


class Wav2Vec2ForAudioClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        self.wav2vec2 = Wav2Vec2Model(config)
        self.dropout = nn.Dropout(0.2)
        self.classifier = nn.Linear(768, 1)

        self.init_weights()

    def forward(self, input_ids, labels):
        outputs = self.wav2vec2(
            input_ids,
            output_attentions=True,
            output_hidden_states=True
        )
        pooled_output = outputs.last_hidden_state[:,0,:]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss_fct = nn.MSELoss()
        loss = loss_fct(logits.view(-1), labels.view(-1))
    
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions
        )

# Training the model for 1 epoch on the popularity scores

Final training log available here: https://wandb.ai/glorf/huggingface/runs/2hkkp2av?workspace=user-glorf

Model can be downloaded from Hf Hub: https://huggingface.co/mbien/fma2vec2popularity

In [6]:
from transformers import Trainer, TrainingArguments

model = Wav2Vec2ForAudioClassification.from_pretrained(
    "/content/drive/MyDrive/musicology-experiment-files/fma2vec", 
).to('cuda')


args = TrainingArguments(
    output_dir="/content/drive/MyDrive/musicology-experiment-files/fma2vec-finetuned-v2",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    eval_steps=5000,
    dataloader_num_workers=32,
    num_train_epochs=1,
    save_steps=5000,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    fp16=True,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=args, 
    train_dataset=train_ds,
    eval_dataset=test_ds
)
trainer.train()

Some weights of Wav2Vec2ForAudioClassification were not initialized from the model checkpoint at /content/drive/MyDrive/musicology-experiment-files/fma2vec and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
wandb: Currently logged in as: glorf (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Step,Training Loss,Validation Loss
5000,0.074600,0.056673
10000,0.063700,0.056438
15000,0.063300,0.060970
20000,0.058300,0.055947
25000,0.054800,0.056436
30000,0.053700,0.057511
35000,0.055000,0.055714
40000,0.061600,0.056468


Broken file found, fixing with prev file sampling


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning

...

Broken file found, fixing with prev file sampling


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Broken file found, fixing with prev file sampling


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


TrainOutput(global_step=43837, training_loss=0.06584673597693164, metrics={'train_runtime': 16294.6998, 'train_samples_per_second': 2.69, 'total_flos': 2.08573752727872e+18, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 8581120, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 416387072, 'train_mem_gpu_alloc_delta': 1168186368, 'train_mem_cpu_peaked_delta': 360787968, 'train_mem_gpu_peaked_delta': 9529502208})